# MoBioFP - Fingerphoto Recognition (Fingertip Object Detection)

In [ ]:
import cv2

from ultralytics import YOLO
from mobiofp.utils import (
    fingertip_enhancement,
    fingertip_thresholding,
    fingerprint_mapping,
    fingerprint_enhancement,
)
from mobiofp.background import BackgroundRemoval
from shared import read_images, show_images

## Define global constants

In [ ]:
SAMPLE_DIR = "../data/raw/samples"
PROCESSED_DIR = "../data/processed/samples/detection"

# Assume the model is already downloaded and placed in the models directory.
# Use one of the following models based on your system architecture.

# MODEL_CHECKPOINT = "../models/fingertip-obj-amd64.pt" # For AMD64
MODEL_CHECKPOINT = "../models/fingertip-obj-arm64.pt"  # For ARM64

## Read sample images

In [ ]:
images, images_titles = read_images(SAMPLE_DIR)
show_images(images, images_titles, fig_size=15, sup_title="Sample Fingerphoto Images")

## Fingertip detection using YOLOv8n pre-trained model

In [ ]:
model = YOLO(MODEL_CHECKPOINT)
model.info()
results = model(images, stream=True, max_det=1)

predicted_images = []
predicted_images_titles = []
bbox_coords = []
fingertip_images = []
fingertip_images_titles = []

for result, title in zip(results, images_titles):
    boxes = result.boxes.xyxy.tolist()
    if not boxes:
        continue
    boxes = [int(coord) for coord in boxes[0]]
    bbox_coords.append(boxes)

    original = result.orig_img
    x1, y1, x2, y2 = boxes
    fingertip = original[y1:y2, x1:x2]
    fingertip_images.append(fingertip)
    fingertip_images_titles.append(title)

    predicted = result.plot()
    predicted_images.append(predicted)
    predicted_images_titles.append(title)

show_images(
    predicted_images, predicted_images_titles, fig_size=15, sup_title="YOLOv8n Fingertip Detection"
)
show_images(fingertip_images, fingertip_images_titles, sup_title="Fingertip Images")

In [ ]:
remover = BackgroundRemoval()
fingertip_masks = [remover.apply(fingertip) for fingertip in fingertip_images]

show_images(fingertip_masks, fingertip_images_titles, sup_title="Fingertip Masks")

## Fingertip Enhancement

In [ ]:
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in fingertip_images]
show_images(
    gray_images, fingertip_images_titles, cmap="gray", sup_title="Grayscale Fingertip Images"
)

In [ ]:
fingertip_enhanced_images = [fingertip_enhancement(image) for image in gray_images]
show_images(
    fingertip_enhanced_images, fingertip_images_titles, sup_title="Fingertip Enhanced Images"
)

## Fingertip Binarization

In [ ]:
fingertip_thresh_images = [
    cv2.bitwise_and(image, image, mask=mask)
    for image, mask in zip(fingertip_enhanced_images, fingertip_masks)
]
fingertip_thresh_images = [
    fingertip_thresholding(image, blockSize=19) for image in fingertip_thresh_images
]
show_images(
    fingertip_thresh_images,
    fingertip_images_titles,
    cmap="gray",
    sup_title="Fingertip Thresholded Images",
)

## Fingertip to Fingeprint Conversion

The function `fingerprint_mapping()` takes an fingertip-enhanced and converts it into a fingerprint image.
It does this by:

- Resizing the image.
- Calculating the local gradient of the image using Sobel filters.
- Calculating the orientation of the ridges in the fingerprint.
- Extracting a region of the image and smoothing it to reduce noise.
- Calculating the x-signature of the region and finding its local maxima to estimate the ridge period.
- Creating a bank of Gabor filters with different orientations.
- Filtering the image with each filter in the bank.
- Assembling the final result by taking the corresponding convolution result for each pixel based on the closest orientation in the Gabor bank.
- Converting the result to grayscale.

In [ ]:
fingerprints, fingerprint_titles = [], []

for image, title in zip(fingertip_thresh_images, fingertip_images_titles):
    fingerprint = fingerprint_mapping(image)
    if fingerprint is not None:
        fingerprints.append(fingerprint)
        fingerprint_titles.append(title)

show_images(fingerprints, fingerprint_titles, sup_title="Fingerprint Images")

In [ ]:
fingerprint_enhanced_images = [fingerprint_enhancement(fingerprint) for fingerprint in fingerprints]

show_images(
    fingerprint_enhanced_images, fingerprint_titles, sup_title="Fingerprint Enhanced Images"
)

## Thinning

In [ ]:
fingerprint_thinning_images = [
    cv2.ximgproc.thinning(image) for image in fingerprint_enhanced_images
]

show_images(
    fingerprint_thinning_images,
    fingerprint_titles,
    sup_title="Fingerprint Thinned Images",
)